In [1]:
import pandas as pd

In [42]:
ksizes = range(5, 31)


def compute_jaccard(set_a, set_b):
    intersection = ced9_hashes.intersection(bcl2_hashes)
    union = ced9_hashes.union(bcl2_hashes)
    return len(intersection) / len(union)


ced9_template = r"../ced9_P41958.fasta.singleton.hp.k{ksize}.scaled1.kmers.csv"
bcl2_template = "../bcl2_P10415.fasta.singleton.hp.k{ksize}.scaled1.kmers.csv"

# for ksize in ksizes:
ksize = 15
ced9 = pd.read_csv(ced9_template.format(ksize=ksize))
bcl2 = pd.read_csv(bcl2_template.format(ksize=ksize))

In [43]:
alphabet_kmers = pd.concat([ced9.kmer_in_alphabet, bcl2.kmer_in_alphabet])
alphabet_kmers

0      hpppphppphpphhh
1      pppphppphpphhhp
2      ppphppphpphhhpp
3      pphppphpphhhppp
4      phppphpphhhpppp
            ...       
220    hphhhhhhphphhhh
221    phhhhhhphphhhhh
222    hhhhhhphphhhhhh
223    hhhhhphphhhhhhp
224    hhhhphphhhhhhpp
Name: kmer_in_alphabet, Length: 491, dtype: object

In [44]:
alphabet_kmers.value_counts()

kmer_in_alphabet
pphhphhphhhhhph    2
phhphhphhhhhphh    2
hhphhphhhhhphhh    2
hphhphhhhhphhhh    2
phhphhhhhphhhhh    2
                  ..
hphppppphhphhph    1
hhphppppphhphhp    1
phhphppppphhphh    1
pphhphppppphhph    1
hhhhphphhhhhhpp    1
Name: count, Length: 486, dtype: int64

In [45]:
ced9_hashes = set(ced9.hashval)
bcl2_hashes = set(bcl2.hashval)
jaccard = len(ced9_hashes.intersection(bcl2_hashes)) / len(
    ced9_hashes.union(bcl2_hashes)
)
jaccard

0.0102880658436214

In [46]:
ced9.head()

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,filename
0,MTRCTADNSLTNPAY,hpppphppphpphhh,7585921404190484870,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
1,TRCTADNSLTNPAYR,pppphppphpphhhp,13419504689273068280,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
2,RCTADNSLTNPAYRR,ppphppphpphhhpp,7765603197291198508,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
3,CTADNSLTNPAYRRR,pphppphpphhhppp,1151751887590685437,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
4,TADNSLTNPAYRRRT,phppphpphhhpppp,6992360843807507041,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta


In [50]:
ced9_in_bcl2 = ced9.query("hashval in @bcl2_hashes")
ced9_in_bcl2

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,filename
162,QCPMSYGRLIGLISF,pphhphhphhhhhph,2474282513327167724,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
163,CPMSYGRLIGLISFG,phhphhphhhhhphh,873697441541549083,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
164,PMSYGRLIGLISFGG,hhphhphhhhhphhh,4214902949673065833,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
165,MSYGRLIGLISFGGF,hphhphhhhhphhhh,5162299627079407232,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta
166,SYGRLIGLISFGGFV,phhphhhhhphhhhh,3128169917091194898,sp|P41958|CED9_CAEEL Apoptosis regulator ced-9...,ced9_P41958.fasta


In [51]:
bcl2_in_ced9 = bcl2.query("hashval in @ced9_hashes")
bcl2_in_ced9

,kmer_in_sequence,kmer_in_alphabet,hashval,read_name,filename
138,RDGVNWGRIVAFFEF,pphhphhphhhhhph,2474282513327167724,sp|P10415|BCL2_HUMAN Apoptosis regulator Bcl-2...,bcl2_P10415.fasta
139,DGVNWGRIVAFFEFG,phhphhphhhhhphh,873697441541549083,sp|P10415|BCL2_HUMAN Apoptosis regulator Bcl-2...,bcl2_P10415.fasta
140,GVNWGRIVAFFEFGG,hhphhphhhhhphhh,4214902949673065833,sp|P10415|BCL2_HUMAN Apoptosis regulator Bcl-2...,bcl2_P10415.fasta
141,VNWGRIVAFFEFGGV,hphhphhhhhphhhh,5162299627079407232,sp|P10415|BCL2_HUMAN Apoptosis regulator Bcl-2...,bcl2_P10415.fasta
142,NWGRIVAFFEFGGVM,phhphhhhhphhhhh,3128169917091194898,sp|P10415|BCL2_HUMAN Apoptosis regulator Bcl-2...,bcl2_P10415.fasta


## I'd like to show a semi-alignment

In [54]:
prev_i = 0
for i, row in ced9_in_bcl2.iterrows():

    bcl2_possible = bcl2_in_ced9.query("hashval == @row.hashval")
    if len(bcl2_possible) == 1:
        bcl2_kmer_to_add = bcl2_possible.iloc[0].kmer_in_sequence
    else:
        print(bcl2_possible)
        print("Too many bcl2 k-mers, skipping")
        continue

    if prev_i < i - 1:
        # New kmer, don't build onto old one
        ced9_kmer = row.kmer_in_sequence
        hp_kmer = row.kmer_in_alphabet
        bcl2_kmer = bcl2_kmer_to_add
    else:
        # Build onto previous kmer
        ced9_kmer += row.kmer_in_sequence[-1]
        hp_kmer += row.kmer_in_alphabet[-1]
        bcl2_kmer += bcl2_kmer_to_add[-1]

    prev_i = i

print(f"{ced9_kmer}\n{hp_kmer}\n{bcl2_kmer}")


QCPMSYGRLIGLISFGGFV
pphhphhphhhhhphhhhh
RDGVNWGRIVAFFEFGGVM
